# Functions

In [1]:
import pandas as pd
import requests
from typing import Dict, List
import re
import json

def get_github_folder_files(username: str, repo_name: str, folder_path: str,
                           file_extension: str = ".csv") -> List[str]:
    """
    GitHub API를 사용해서 특정 폴더의 파일 목록을 가져오는 함수

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")
    file_extension : str
        필터링할 파일 확장자 (기본값: ".csv")

    Returns:
    --------
    List[str]
        해당 폴더의 CSV 파일명 목록
    """

    # GitHub API URL
    api_url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"

    try:
        response = requests.get(api_url)
        response.raise_for_status()

        files_data = response.json()

        # CSV 파일만 필터링
        csv_files = []
        for file_info in files_data:
            if file_info['type'] == 'file' and file_info['name'].endswith(file_extension):
                csv_files.append(file_info['name'])

        return sorted(csv_files)

    except requests.exceptions.RequestException as e:
        print(f"❌ GitHub API 요청 실패: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"❌ JSON 파싱 실패: {e}")
        return []


def create_variable_name_from_filename(filename: str) -> str:
    """
    파일명에서 변수명을 생성하는 함수
    temp1_governor_지역명_숫자.csv → g숫자_지역명

    Parameters:
    -----------
    filename : str
        파일명 (예: "temp1_governor_busan_3.csv", "temp1_governor_seoul_4.csv")

    Returns:
    --------
    str
        변수명 (예: "g3_busan", "g4_seoul")
    """

    # temp1_governor_지역명_숫자.csv → g숫자_지역명
    match = re.search(r'temp1_governor_([^_]+)_(\d+)\.csv', filename)
    if match:
        region = match.group(1)
        number = match.group(2)
        return f"g{number}_{region}"

    # 매칭되지 않으면 기본적으로 파일명 정리해서 반환
    base_name = filename.replace('.csv', '').replace('temp1_', '')
    return base_name


def load_all_csvs_from_github_folder(username: str, repo_name: str, folder_path: str) -> Dict[str, pd.DataFrame]:
    """
    GitHub 폴더에서 모든 CSV 파일을 자동으로 읽어와서 DataFrame 딕셔너리로 반환

    Parameters:
    -----------
    username : str
        GitHub 사용자명
    repo_name : str
        레포지토리 이름 (예: "korean-elections")
    folder_path : str
        폴더 경로 (예: "temp/v1_g/3rd_2002")

    Returns:
    --------
    Dict[str, pd.DataFrame]
        변수명을 키로 하고 DataFrame을 값으로 하는 딕셔너리
    """

    # 1. GitHub API로 파일 목록 가져오기
    csv_files = get_github_folder_files(username, repo_name, folder_path)

    if not csv_files:
        print("❌ CSV 파일을 찾을 수 없습니다.")
        return {}

    # 2. 각 CSV 파일 읽어오기
    base_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/refs/heads/main/{folder_path}/"
    dataframes = {}

    for filename in csv_files:
        try:
            # URL 생성
            url = base_url + filename

            # 변수명 생성
            var_name = create_variable_name_from_filename(filename)

            # CSV 읽기
            df = pd.read_csv(url)

            dataframes[var_name] = df
            print(f"✅ {filename} → {var_name}")

        except Exception as e:
            print(f"❌ {filename} 읽기 실패: {e}")

    return dataframes


def create_individual_variables(dataframes: Dict[str, pd.DataFrame], globals_dict):
    """
    딕셔너리로 저장된 DataFrame들을 개별 변수로 생성

    Parameters:
    -----------
    dataframes : Dict[str, pd.DataFrame]
        load_all_csvs_from_github_folder()의 반환값
    globals_dict : dict
        변수를 저장할 전역 네임스페이스 (globals()를 전달)
    """

    for var_name, df in dataframes.items():
        globals_dict[var_name] = df
        print(f"변수 생성: {var_name}")


# # 사용 예시
# def example_usage():
#     """사용 예시"""

#     print("=== GitHub 폴더 자동 CSV 로더 ===\n")

#     print("사용법:")
#     print("""
#     username = "your-username"
#     repo_name = "korean-elections"
#     folder_path = "temp/v1_g/3rd_2002"

#     # 모든 CSV 파일을 자동으로 읽어와서 딕셔너리로 저장
#     election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

#     # 방법 1: 딕셔너리로 접근
#     busan_df = election_data['g3_busan']
#     seoul_df = election_data['g3_seoul']

#     # 방법 2: 개별 변수로 생성
#     create_individual_variables(election_data, globals())
#     # 이제 g3_busan, g3_seoul 등을 직접 사용 가능
#     """)

#     print("\n다양한 번호 지원:")
#     print("""
#     # 3회차: temp1_governor_busan_3.csv → g3_busan
#     election_3rd = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/3rd_2002")

#     # 4회차: temp1_governor_seoul_4.csv → g4_seoul
#     election_4th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/4th_2006")

#     # 5회차: temp1_governor_daegu_5.csv → g5_daegu
#     election_5th = load_all_csvs_from_github_folder(username, repo_name, "temp/v1_g/5th_2010")
#     """)


# if __name__ == "__main__":
#     example_usage()

# ### 간단한 사용법

# # 한 줄로 모든 CSV 읽기
# username = "your-github-username"
# repo_name = "korean-elections"
# folder_path = "temp/v1_g/3rd_2002"

# election_data = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# # 사용하기
# busan_df = election_data['g3_busan']
# seoul_df = election_data['g3_seoul']

# # 또는 개별 변수로 생성
# create_individual_variables(election_data, globals())
# # 이제 g3_busan, g3_seoul 직접 사용 가능

In [2]:
import pandas as pd

def show_column_info(df):
    """
    데이터프레임의 열 이름과 인덱스를 확인하는 함수
    """
    print("=== 열 이름과 인덱스 ===")
    for idx, col_name in enumerate(df.columns):
        print(f"{idx}: {col_name}")
    print(f"\n총 열 개수: {len(df.columns)}")
    print(f"총 행 개수: {len(df)}")

def rename_columns_by_index(df, index_or_indices, new_name_or_names):
    """
    인덱스를 통해 열 이름을 변경하는 함수

    Parameters:
    - df: pandas DataFrame
    - index_or_indices: 단일 인덱스(int) 또는 인덱스 리스트([int, int, ...])
    - new_name_or_names: 단일 새 이름(str) 또는 새 이름 리스트([str, str, ...])

    Returns:
    - 열 이름이 변경된 DataFrame
    """
    # 단일 값을 리스트로 변환
    if isinstance(index_or_indices, int):
        indices = [index_or_indices]
    else:
        indices = index_or_indices

    if isinstance(new_name_or_names, str):
        new_names = [new_name_or_names]
    else:
        new_names = new_name_or_names

    # 길이 검증
    if len(indices) != len(new_names):
        raise ValueError(f"인덱스 개수({len(indices)})와 새 이름 개수({len(new_names)})가 일치하지 않습니다.")

    # 인덱스 유효성 검사
    for idx in indices:
        if idx < 0 or idx >= len(df.columns):
            raise ValueError(f"유효하지 않은 인덱스: {idx}. 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 열 이름 변경
    rename_dict = {}
    for idx, new_name in zip(indices, new_names):
        old_name = df.columns[idx]
        rename_dict[old_name] = new_name
        print(f"인덱스 {idx}: '{old_name}' -> '{new_name}'")

    result_df = result_df.rename(columns=rename_dict)

    print(f"\n=== 열 이름 변경 완료 ===")
    print(f"변경된 열 개수: {len(rename_dict)}")

    return result_df

def add_sum_column(df, start_idx, end_idx, new_column_name):
    """
    연속된 인덱스 범위의 열들을 합산하여 새로운 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - start_idx: 시작 인덱스
    - end_idx: 끝 인덱스 (포함)
    - new_column_name: 새로 추가할 열의 이름

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    # 유효성 검사
    if start_idx < 0 or end_idx >= len(df.columns) or start_idx > end_idx:
        raise ValueError(f"유효하지 않은 인덱스 범위: ({start_idx}, {end_idx}). 유효 범위: 0 ~ {len(df.columns) - 1}")

    # 합산할 열 이름들 추출
    columns_to_sum = df.columns[start_idx:end_idx + 1]

    print(f"\n=== 합산할 열들 (인덱스 {start_idx}~{end_idx}) ===")
    for idx, col_name in enumerate(columns_to_sum):
        print(f"{start_idx + idx}: {col_name}")

    # 데이터프레임 복사
    result_df = df.copy()

    # 지정된 열들을 합산하여 새로운 열 추가
    # 숫자가 아닌 값은 0으로 처리
    result_df[new_column_name] = df[columns_to_sum].select_dtypes(include=['number']).sum(axis=1, skipna=True)

    print(f"\n=== 새로운 열 추가 완료 ===")
    print(f"새로운 열 이름: {new_column_name}")
    print(f"총 열 개수: {len(result_df.columns)}")

    return result_df

def add_zero_column(df, column_name):
    """
    DataFrame 마지막에 모든 원소가 0(int)인 열을 추가하는 함수

    Parameters:
    - df: pandas DataFrame
    - column_name: 추가할 열의 이름 (str)

    Returns:
    - 새로운 열이 추가된 DataFrame
    """
    result_df = df.copy()
    result_df[column_name] = 0
    return result_df

# # 사용 예시
# if __name__ == "__main__":
#     # CSV 파일 읽기
#     df = pd.read_csv('temp1_president_14.csv')

#     # 열 정보 확인
#     show_column_info(df)

#     # 예시 1: 단일 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 1: 단일 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df, 0, '광역시도')

#     # 예시 2: 복수 열 이름 변경
#     print("\n" + "="*50)
#     print("예시 2: 복수 열 이름 변경")
#     print("="*50)
#     df_renamed = rename_columns_by_index(df_renamed, [1, 2], ['시군구', '유권자수'])

#     # 예시 3: 인덱스 4~7 열들 합산 (득표수 1~4번 후보)
#     print("\n" + "="*50)
#     print("예시 3: 열 합산")
#     print("="*50)
#     result_df = add_sum_column(df_renamed, 4, 7, '상위4후보합계')

#     # 변경된 열 정보 확인
#     print("\n" + "="*50)
#     print("변경된 열 정보")
#     print("="*50)
#     show_column_info(result_df)

#     # 결과 확인 (첫 번째 행)
#     print("\n=== 첫 번째 행 결과 ===")
#     first_row = result_df.iloc[0]
#     print(f"광역시도: {first_row['광역시도']}")
#     print(f"시군구: {first_row['시군구']}")
#     print(f"유권자수: {first_row['유권자수']}")
#     print(f"득표수_1_민주자유당_김영삼: {first_row['득표수_1_민주자유당_김영삼']}")
#     print(f"득표수_2_민주당_김대중: {first_row['득표수_2_민주당_김대중']}")
#     print(f"득표수_3_통일국민당_정주영: {first_row['득표수_3_통일국민당_정주영']}")
#     print(f"득표수_4_새한국당_이종찬: {first_row['득표수_4_새한국당_이종찬']}")
#     print(f"상위4후보합계: {first_row['상위4후보합계']}")

#     # 검증: 수동 계산과 비교
#     manual_sum = (first_row['득표수_1_민주자유당_김영삼'] +
#                   first_row['득표수_2_민주당_김대중'] +
#                   first_row['득표수_3_통일국민당_정주영'] +
#                   first_row['득표수_4_새한국당_이종찬'])

#     print(f"\n수동 계산 결과: {manual_sum}")
#     print(f"함수 계산 결과: {first_row['상위4후보합계']}")
#     print(f"계산 일치: {manual_sum == first_row['상위4후보합계']}")

#     # 결과 데이터프레임 출력 (상위 5행)
#     print("\n=== 결과 데이터프레임 (상위 5행) ===")
#     print(result_df.head())

# Data (v1, 3rd_2002)

In [3]:
# 한 줄로 모든 CSV 읽기
username = "sw1kwon"
repo_name = "korean-elections"
folder_path = "temp/v1_g/3rd_2002"

governor_3 = load_all_csvs_from_github_folder(username, repo_name, folder_path)

# 개별 변수로 생성
create_individual_variables(governor_3, globals())
# 이제 g3_busan, g3_seoul 직접 사용 가능

✅ temp1_governor_busan_3.csv → g3_busan
✅ temp1_governor_chungbuk_3.csv → g3_chungbuk
✅ temp1_governor_chungnam_3.csv → g3_chungnam
✅ temp1_governor_daegu_3.csv → g3_daegu
✅ temp1_governor_daejeon_3.csv → g3_daejeon
✅ temp1_governor_gangwon_3.csv → g3_gangwon
✅ temp1_governor_gwangju_3.csv → g3_gwangju
✅ temp1_governor_gyeongbuk_3.csv → g3_gyeongbuk
✅ temp1_governor_gyeonggi_3.csv → g3_gyeonggi
✅ temp1_governor_gyeongnam_3.csv → g3_gyeongnam
✅ temp1_governor_incheon_3.csv → g3_incheon
✅ temp1_governor_jeju_3.csv → g3_jeju
✅ temp1_governor_jeonbuk_3.csv → g3_jeonbuk
✅ temp1_governor_jeonnam_3.csv → g3_jeonnam
✅ temp1_governor_seoul_3.csv → g3_seoul
✅ temp1_governor_ulsan_3.csv → g3_ulsan
변수 생성: g3_busan
변수 생성: g3_chungbuk
변수 생성: g3_chungnam
변수 생성: g3_daegu
변수 생성: g3_daejeon
변수 생성: g3_gangwon
변수 생성: g3_gwangju
변수 생성: g3_gyeongbuk
변수 생성: g3_gyeonggi
변수 생성: g3_gyeongnam
변수 생성: g3_incheon
변수 생성: g3_jeju
변수 생성: g3_jeonbuk
변수 생성: g3_jeonnam
변수 생성: g3_seoul
변수 생성: g3_ulsan


## Seoul

In [4]:
g3_seoul.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이명박,득표수_2_새천년민주당_김민석,득표수_3_녹색평화당_임삼진,득표수_4_민주노동당_이문옥,득표수_5_사회당_원용수,득표수_6_무소속_이경희,득표수_계,무효투표수,기권수
0,서울특별시,합계,7665343,3510898,1819057,1496754,28034,87965,12982,34313,3479105,31793,4154445
1,서울특별시,종로구,141157,72241,37592,30353,584,1928,305,689,71451,790,68916
2,서울특별시,중구,110996,56999,28220,25799,410,1167,190,516,56302,697,53997
3,서울특별시,용산구,186438,89215,47878,36435,686,2154,338,823,88314,901,97223
4,서울특별시,성동구,259275,122030,60188,54742,965,3274,461,1184,120814,1216,137245


In [5]:
show_column_info(g3_seoul)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_이명박
5: 득표수_2_새천년민주당_김민석
6: 득표수_3_녹색평화당_임삼진
7: 득표수_4_민주노동당_이문옥
8: 득표수_5_사회당_원용수
9: 득표수_6_무소속_이경희
10: 득표수_계
11: 무효투표수
12: 기권수

총 열 개수: 13
총 행 개수: 26


### v2.1

In [6]:
g3_seoul_21 = rename_columns_by_index(g3_seoul, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_이명박' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_김민석' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [7]:
g3_seoul_21 = add_sum_column(g3_seoul_21, 6, 9, '그외정당')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_3_녹색평화당_임삼진
7: 득표수_4_민주노동당_이문옥
8: 득표수_5_사회당_원용수
9: 득표수_6_무소속_이경희

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [8]:
g3_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_녹색평화당_임삼진,득표수_4_민주노동당_이문옥,득표수_5_사회당_원용수,득표수_6_무소속_이경희,득표수_계,무효투표수,기권수,그외정당
0,서울특별시,합계,7665343,3510898,1819057,1496754,28034,87965,12982,34313,3479105,31793,4154445,163294
1,서울특별시,종로구,141157,72241,37592,30353,584,1928,305,689,71451,790,68916,3506
2,서울특별시,중구,110996,56999,28220,25799,410,1167,190,516,56302,697,53997,2283


In [9]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_seoul_21 = g3_seoul_21[selected_columns].copy()

g3_seoul_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,서울특별시,합계,7665343,3510898,1819057,1496754,163294,3479105,31793,4154445
1,서울특별시,종로구,141157,72241,37592,30353,3506,71451,790,68916
2,서울특별시,중구,110996,56999,28220,25799,2283,56302,697,53997


In [10]:
g3_seoul_21.to_csv("temp2_1_governor_seoul_3.csv", index=False, encoding="utf-8-sig")

### v2.2

In [11]:
g3_seoul_22 = rename_columns_by_index(g3_seoul, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_이명박' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_김민석' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [12]:
g3_seoul_22 = add_sum_column(g3_seoul_22, 6, 8, '그외정당')
g3_seoul_22 = add_sum_column(g3_seoul_22, 9, 9, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_녹색평화당_임삼진
7: 득표수_4_민주노동당_이문옥
8: 득표수_5_사회당_원용수

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14

=== 합산할 열들 (인덱스 9~9) ===
9: 득표수_6_무소속_이경희

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 15


In [13]:
g3_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_녹색평화당_임삼진,득표수_4_민주노동당_이문옥,득표수_5_사회당_원용수,득표수_6_무소속_이경희,득표수_계,무효투표수,기권수,그외정당,무소속
0,서울특별시,합계,7665343,3510898,1819057,1496754,28034,87965,12982,34313,3479105,31793,4154445,128981,34313
1,서울특별시,종로구,141157,72241,37592,30353,584,1928,305,689,71451,790,68916,2817,689
2,서울특별시,중구,110996,56999,28220,25799,410,1167,190,516,56302,697,53997,1767,516


In [14]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_seoul_22 = g3_seoul_22[selected_columns].copy()

g3_seoul_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,서울특별시,합계,7665343,3510898,1819057,1496754,128981,34313,3479105,31793,4154445
1,서울특별시,종로구,141157,72241,37592,30353,2817,689,71451,790,68916
2,서울특별시,중구,110996,56999,28220,25799,1767,516,56302,697,53997


In [15]:
g3_seoul_22.to_csv("temp2_2_governor_seoul_3.csv", index=False, encoding="utf-8-sig")

## Busan

In [16]:
g3_busan.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상영,득표수_2_새천년민주당_한이헌,득표수_3_민주노동당_김석준,득표수_계,무효투표수,기권수
0,부산광역시,합계,2784721,1163040,729589,221938,192594,1144121,18919,1621681
1,부산광역시,중구,45490,23688,15669,4039,3487,23195,493,21802
2,부산광역시,서구,116858,47313,31336,8397,6594,46327,986,69545
3,부산광역시,동구,97322,44749,29853,7808,6247,43908,841,52573
4,부산광역시,영도구,136484,59273,37299,12417,8545,58261,1012,77211


In [17]:
show_column_info(g3_busan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_안상영
5: 득표수_2_새천년민주당_한이헌
6: 득표수_3_민주노동당_김석준
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 17


### v2.1

In [18]:
g3_busan_21 = rename_columns_by_index(g3_busan, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_안상영' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_한이헌' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [19]:
g3_busan_21 = add_sum_column(g3_busan_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주노동당_김석준

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [20]:
g3_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_김석준,득표수_계,무효투표수,기권수,그외정당
0,부산광역시,합계,2784721,1163040,729589,221938,192594,1144121,18919,1621681,192594
1,부산광역시,중구,45490,23688,15669,4039,3487,23195,493,21802,3487
2,부산광역시,서구,116858,47313,31336,8397,6594,46327,986,69545,6594


In [21]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_busan_21 = g3_busan_21[selected_columns].copy()

g3_busan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,부산광역시,합계,2784721,1163040,729589,221938,192594,1144121,18919,1621681
1,부산광역시,중구,45490,23688,15669,4039,3487,23195,493,21802
2,부산광역시,서구,116858,47313,31336,8397,6594,46327,986,69545


In [22]:
g3_busan_21.to_csv("temp2_1_governor_busan_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [23]:
g3_busan_22 = rename_columns_by_index(g3_busan, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_안상영' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_한이헌' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [24]:
g3_busan_22 = add_sum_column(g3_busan_22, 6, 6, '그외정당')
g3_busan_22 = add_zero_column(g3_busan_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주노동당_김석준

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [25]:
g3_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_김석준,득표수_계,무효투표수,기권수,그외정당,무소속
0,부산광역시,합계,2784721,1163040,729589,221938,192594,1144121,18919,1621681,192594,0
1,부산광역시,중구,45490,23688,15669,4039,3487,23195,493,21802,3487,0
2,부산광역시,서구,116858,47313,31336,8397,6594,46327,986,69545,6594,0


In [26]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_busan_22 = g3_busan_22[selected_columns].copy()

g3_busan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,부산광역시,합계,2784721,1163040,729589,221938,192594,0,1144121,18919,1621681
1,부산광역시,중구,45490,23688,15669,4039,3487,0,23195,493,21802
2,부산광역시,서구,116858,47313,31336,8397,6594,0,46327,986,69545


In [27]:
g3_busan_22.to_csv("temp2_2_governor_busan_3.csv", index=False, encoding="utf-8-sig")

## Daegu

In [28]:
g3_daegu.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_조해녕,득표수_3_무소속_이재용,득표수_계,무효투표수,기권수
0,대구광역시,합계,1814278,751994,452943,287396,740339,11655,1062284
1,대구광역시,중구,70409,34423,20807,13020,33827,596,35986
2,대구광역시,동구,250562,105543,66098,37617,103715,1828,145019
3,대구광역시,서구,206981,84479,53305,29549,82854,1625,122502
4,대구광역시,남구,148146,64865,31249,32866,64115,750,83281


In [29]:
show_column_info(g3_daegu)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_조해녕
5: 득표수_3_무소속_이재용
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 9


### v2.1 (XXX)

In [30]:
g3_daegu_21 = rename_columns_by_index(g3_daegu, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_조해녕' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [31]:
g3_daegu_21 = add_sum_column(g3_daegu_21, 5, 5, '그외정당')
g3_daegu_21 = add_zero_column(g3_daegu_21, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_무소속_이재용

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [32]:
g3_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_무소속_이재용,득표수_계,무효투표수,기권수,그외정당,진보정당
0,대구광역시,합계,1814278,751994,452943,287396,740339,11655,1062284,287396,0
1,대구광역시,중구,70409,34423,20807,13020,33827,596,35986,13020,0
2,대구광역시,동구,250562,105543,66098,37617,103715,1828,145019,37617,0


In [33]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_daegu_21 = g3_daegu_21[selected_columns].copy()

g3_daegu_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대구광역시,합계,1814278,751994,452943,0,287396,740339,11655,1062284
1,대구광역시,중구,70409,34423,20807,0,13020,33827,596,35986
2,대구광역시,동구,250562,105543,66098,0,37617,103715,1828,145019


In [34]:
g3_daegu_21.to_csv("temp2_1_governor_daegu_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, XX)

In [35]:
g3_daegu_22 = rename_columns_by_index(g3_daegu, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_조해녕' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [36]:
g3_daegu_22 = add_sum_column(g3_daegu_22, 5, 5, '무소속')
g3_daegu_22 = add_zero_column(g3_daegu_22, '진보정당')
g3_daegu_22 = add_zero_column(g3_daegu_22, '그외정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_무소속_이재용

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 10


In [37]:
g3_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_무소속_이재용,득표수_계,무효투표수,기권수,무소속,진보정당,그외정당
0,대구광역시,합계,1814278,751994,452943,287396,740339,11655,1062284,287396,0,0
1,대구광역시,중구,70409,34423,20807,13020,33827,596,35986,13020,0,0
2,대구광역시,동구,250562,105543,66098,37617,103715,1828,145019,37617,0,0


In [38]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_daegu_22 = g3_daegu_22[selected_columns].copy()

g3_daegu_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대구광역시,합계,1814278,751994,452943,0,0,287396,740339,11655,1062284
1,대구광역시,중구,70409,34423,20807,0,0,13020,33827,596,35986
2,대구광역시,동구,250562,105543,66098,0,0,37617,103715,1828,145019


In [39]:
g3_daegu_22.to_csv("temp2_2_governor_daegu_3.csv", index=False, encoding="utf-8-sig")

## Incheon

In [40]:
g3_incheon.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_안상수,득표수_2_새천년민주당_박상은,득표수_3_녹색평화당_신맹순,득표수_4_민주노동당_김창한,득표수_5_사회당_김영규,득표수_계,무효투표수,기권수
0,인천광역시,합계,1809907,711602,393932,225210,29473,35234,17404,701253,10349,1098305
1,인천광역시,중구,56476,27197,15627,8295,1154,987,599,26662,535,29279
2,인천광역시,동구,57370,28452,16071,8484,971,1484,806,27816,636,28918
3,인천광역시,남구,317337,124830,71928,37404,5341,5028,3276,122977,1853,192507
4,인천광역시,연수구,174043,67745,40332,19621,2711,2590,1681,66935,810,106298


In [41]:
show_column_info(g3_incheon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_안상수
5: 득표수_2_새천년민주당_박상은
6: 득표수_3_녹색평화당_신맹순
7: 득표수_4_민주노동당_김창한
8: 득표수_5_사회당_김영규
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 11


### v2.1

In [42]:
g3_incheon_21 = rename_columns_by_index(g3_incheon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_안상수' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_박상은' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [43]:
g3_incheon_21 = add_sum_column(g3_incheon_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_녹색평화당_신맹순
7: 득표수_4_민주노동당_김창한
8: 득표수_5_사회당_김영규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [44]:
g3_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_녹색평화당_신맹순,득표수_4_민주노동당_김창한,득표수_5_사회당_김영규,득표수_계,무효투표수,기권수,그외정당
0,인천광역시,합계,1809907,711602,393932,225210,29473,35234,17404,701253,10349,1098305,82111
1,인천광역시,중구,56476,27197,15627,8295,1154,987,599,26662,535,29279,2740
2,인천광역시,동구,57370,28452,16071,8484,971,1484,806,27816,636,28918,3261


In [45]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_incheon_21 = g3_incheon_21[selected_columns].copy()

g3_incheon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,인천광역시,합계,1809907,711602,393932,225210,82111,701253,10349,1098305
1,인천광역시,중구,56476,27197,15627,8295,2740,26662,535,29279
2,인천광역시,동구,57370,28452,16071,8484,3261,27816,636,28918


In [46]:
g3_incheon_21.to_csv("temp2_1_governor_incheon_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [47]:
g3_incheon_22 = rename_columns_by_index(g3_incheon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_안상수' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_박상은' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [48]:
g3_incheon_22 = add_sum_column(g3_incheon_22, 6, 8, '그외정당')
g3_incheon_22 = add_zero_column(g3_incheon_22, '무소속')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_녹색평화당_신맹순
7: 득표수_4_민주노동당_김창한
8: 득표수_5_사회당_김영규

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [49]:
g3_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_녹색평화당_신맹순,득표수_4_민주노동당_김창한,득표수_5_사회당_김영규,득표수_계,무효투표수,기권수,그외정당,무소속
0,인천광역시,합계,1809907,711602,393932,225210,29473,35234,17404,701253,10349,1098305,82111,0
1,인천광역시,중구,56476,27197,15627,8295,1154,987,599,26662,535,29279,2740,0
2,인천광역시,동구,57370,28452,16071,8484,971,1484,806,27816,636,28918,3261,0


In [50]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_incheon_22 = g3_incheon_22[selected_columns].copy()

g3_incheon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,인천광역시,합계,1809907,711602,393932,225210,82111,0,701253,10349,1098305
1,인천광역시,중구,56476,27197,15627,8295,2740,0,26662,535,29279
2,인천광역시,동구,57370,28452,16071,8484,3261,0,27816,636,28918


In [51]:
g3_incheon_22.to_csv("temp2_2_governor_incheon_3.csv", index=False, encoding="utf-8-sig")

## Gwangju

In [52]:
g3_gwangju.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이환의,득표수_2_새천년민주당_박광태,득표수_3_민주노동당_박종현,득표수_4_무소속_정구선,득표수_5_무소속_정동년,득표수_6_무소속_정호선,득표수_계,무효투표수,기권수
0,광주광역시,합계,954481,403821,43695,185938,29427,9877,107415,20796,397148,6673,550660
1,광주광역시,동구,89770,42924,6436,20363,2593,1171,9445,2144,42152,772,46846
2,광주광역시,서구,204779,85482,10740,39589,5565,1778,22435,4219,84326,1156,119297
3,광주광역시,남구,159676,71438,7523,34966,3652,1647,18855,3679,70322,1116,88238
4,광주광역시,북구,324452,128528,12760,55377,9123,3568,38680,7156,126664,1864,195924


In [53]:
show_column_info(g3_gwangju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_이환의
5: 득표수_2_새천년민주당_박광태
6: 득표수_3_민주노동당_박종현
7: 득표수_4_무소속_정구선
8: 득표수_5_무소속_정동년
9: 득표수_6_무소속_정호선
10: 득표수_계
11: 무효투표수
12: 기권수

총 열 개수: 13
총 행 개수: 6


### v2.1

In [54]:
g3_gwangju_21 = rename_columns_by_index(g3_gwangju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_이환의' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_박광태' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [55]:
g3_gwangju_21 = add_sum_column(g3_gwangju_21, 6, 9, '그외정당')


=== 합산할 열들 (인덱스 6~9) ===
6: 득표수_3_민주노동당_박종현
7: 득표수_4_무소속_정구선
8: 득표수_5_무소속_정동년
9: 득표수_6_무소속_정호선

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14


In [56]:
g3_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_박종현,득표수_4_무소속_정구선,득표수_5_무소속_정동년,득표수_6_무소속_정호선,득표수_계,무효투표수,기권수,그외정당
0,광주광역시,합계,954481,403821,43695,185938,29427,9877,107415,20796,397148,6673,550660,167515
1,광주광역시,동구,89770,42924,6436,20363,2593,1171,9445,2144,42152,772,46846,15353
2,광주광역시,서구,204779,85482,10740,39589,5565,1778,22435,4219,84326,1156,119297,33997


In [57]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_gwangju_21 = g3_gwangju_21[selected_columns].copy()

g3_gwangju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,광주광역시,합계,954481,403821,43695,185938,167515,397148,6673,550660
1,광주광역시,동구,89770,42924,6436,20363,15353,42152,772,46846
2,광주광역시,서구,204779,85482,10740,39589,33997,84326,1156,119297


In [58]:
g3_gwangju_21.to_csv("temp2_1_governor_gwangju_3.csv", index=False, encoding="utf-8-sig")

### v2.2

In [59]:
g3_gwangju_22 = rename_columns_by_index(g3_gwangju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_이환의' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_박광태' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [60]:
g3_gwangju_22 = add_sum_column(g3_gwangju_22, 6, 6, '그외정당')
g3_gwangju_22 = add_sum_column(g3_gwangju_22, 7, 9, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주노동당_박종현

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 14

=== 합산할 열들 (인덱스 7~9) ===
7: 득표수_4_무소속_정구선
8: 득표수_5_무소속_정동년
9: 득표수_6_무소속_정호선

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 15


In [61]:
g3_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_박종현,득표수_4_무소속_정구선,득표수_5_무소속_정동년,득표수_6_무소속_정호선,득표수_계,무효투표수,기권수,그외정당,무소속
0,광주광역시,합계,954481,403821,43695,185938,29427,9877,107415,20796,397148,6673,550660,29427,138088
1,광주광역시,동구,89770,42924,6436,20363,2593,1171,9445,2144,42152,772,46846,2593,12760
2,광주광역시,서구,204779,85482,10740,39589,5565,1778,22435,4219,84326,1156,119297,5565,28432


In [62]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_gwangju_22 = g3_gwangju_22[selected_columns].copy()

g3_gwangju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,광주광역시,합계,954481,403821,43695,185938,29427,138088,397148,6673,550660
1,광주광역시,동구,89770,42924,6436,20363,2593,12760,42152,772,46846
2,광주광역시,서구,204779,85482,10740,39589,5565,28432,84326,1156,119297


In [63]:
g3_gwangju_22.to_csv("temp2_2_governor_gwangju_3.csv", index=False, encoding="utf-8-sig")

## Daejeon

In [64]:
g3_daejeon.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_염홍철,득표수_3_자유민주연합_홍선기,득표수_4_무소속_김헌태,득표수_5_무소속_정하용,득표수_계,무효투표수,기권수
0,대전광역시,합계,987180,417809,191832,165426,15780,38445,411483,6326,569371
1,대전광역시,동구,181210,76220,32969,32035,3066,6623,74693,1527,104990
2,대전광역시,중구,195396,83818,37980,33683,3105,7742,82510,1308,111578
3,대전광역시,서구,333834,140783,67939,53426,4711,13131,139207,1576,193051
4,대전광역시,유성구,119056,52548,24855,19643,2112,5102,51712,836,66508


In [65]:
show_column_info(g3_daejeon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_염홍철
5: 득표수_3_자유민주연합_홍선기
6: 득표수_4_무소속_김헌태
7: 득표수_5_무소속_정하용
8: 득표수_계
9: 무효투표수
10: 기권수

총 열 개수: 11
총 행 개수: 6


### v2.1 (XXX)

In [66]:
g3_daejeon_21 = rename_columns_by_index(g3_daejeon, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_염홍철' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [67]:
g3_daejeon_21 = add_sum_column(g3_daejeon_21, 5, 7, '그외정당')
g3_daejeon_21 = add_zero_column(g3_daejeon_21, '진보정당')


=== 합산할 열들 (인덱스 5~7) ===
5: 득표수_3_자유민주연합_홍선기
6: 득표수_4_무소속_김헌태
7: 득표수_5_무소속_정하용

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12


In [68]:
g3_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_자유민주연합_홍선기,득표수_4_무소속_김헌태,득표수_5_무소속_정하용,득표수_계,무효투표수,기권수,그외정당,진보정당
0,대전광역시,합계,987180,417809,191832,165426,15780,38445,411483,6326,569371,219651,0
1,대전광역시,동구,181210,76220,32969,32035,3066,6623,74693,1527,104990,41724,0
2,대전광역시,중구,195396,83818,37980,33683,3105,7742,82510,1308,111578,44530,0


In [69]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_daejeon_21 = g3_daejeon_21[selected_columns].copy()

g3_daejeon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,대전광역시,합계,987180,417809,191832,0,219651,411483,6326,569371
1,대전광역시,동구,181210,76220,32969,0,41724,74693,1527,104990
2,대전광역시,중구,195396,83818,37980,0,44530,82510,1308,111578


In [70]:
g3_daejeon_21.to_csv("temp2_1_governor_daejeon_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX)

In [71]:
g3_daejeon_22 = rename_columns_by_index(g3_daejeon, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_염홍철' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [72]:
g3_daejeon_22 = add_sum_column(g3_daejeon_22, 5, 5, '그외정당')
g3_daejeon_22 = add_sum_column(g3_daejeon_22, 6, 7, '무소속')
g3_daejeon_22 = add_zero_column(g3_daejeon_22, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_자유민주연합_홍선기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 12

=== 합산할 열들 (인덱스 6~7) ===
6: 득표수_4_무소속_김헌태
7: 득표수_5_무소속_정하용

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 13


In [73]:
g3_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_자유민주연합_홍선기,득표수_4_무소속_김헌태,득표수_5_무소속_정하용,득표수_계,무효투표수,기권수,그외정당,무소속,진보정당
0,대전광역시,합계,987180,417809,191832,165426,15780,38445,411483,6326,569371,165426,54225,0
1,대전광역시,동구,181210,76220,32969,32035,3066,6623,74693,1527,104990,32035,9689,0
2,대전광역시,중구,195396,83818,37980,33683,3105,7742,82510,1308,111578,33683,10847,0


In [74]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_daejeon_22 = g3_daejeon_22[selected_columns].copy()

g3_daejeon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,대전광역시,합계,987180,417809,191832,0,165426,54225,411483,6326,569371
1,대전광역시,동구,181210,76220,32969,0,32035,9689,74693,1527,104990
2,대전광역시,중구,195396,83818,37980,0,33683,10847,82510,1308,111578


In [75]:
g3_daejeon_22.to_csv("temp2_2_governor_daejeon_3.csv", index=False, encoding="utf-8-sig")

## Ulsan

In [76]:
g3_ulsan.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박맹우,득표수_3_민주노동당_송철호,득표수_4_사회당_안승천,득표수_계,무효투표수,기권수
0,울산광역시,합계,722806,378121,197772,162546,12329,372647,5474,344685
1,울산광역시,중구,162972,81207,45410,32569,2113,80092,1115,81765
2,울산광역시,남구,234702,108730,63805,40808,2996,107609,1121,125972
3,울산광역시,동구,125305,70935,28200,39071,2693,69964,971,54370
4,울산광역시,북구,79943,47783,20174,25593,1424,47191,592,32160


In [77]:
show_column_info(g3_ulsan)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_박맹우
5: 득표수_3_민주노동당_송철호
6: 득표수_4_사회당_안승천
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 6


### v2.1 (XXX)

In [78]:
g3_ulsan_21 = rename_columns_by_index(g3_ulsan, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_박맹우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [79]:
g3_ulsan_21 = add_sum_column(g3_ulsan_21, 5, 6, '그외정당')
g3_ulsan_21 = add_zero_column(g3_ulsan_21, '진보정당')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_3_민주노동당_송철호
6: 득표수_4_사회당_안승천

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [80]:
g3_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_민주노동당_송철호,득표수_4_사회당_안승천,득표수_계,무효투표수,기권수,그외정당,진보정당
0,울산광역시,합계,722806,378121,197772,162546,12329,372647,5474,344685,174875,0
1,울산광역시,중구,162972,81207,45410,32569,2113,80092,1115,81765,34682,0
2,울산광역시,남구,234702,108730,63805,40808,2996,107609,1121,125972,43804,0


In [81]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_ulsan_21 = g3_ulsan_21[selected_columns].copy()

g3_ulsan_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,울산광역시,합계,722806,378121,197772,0,174875,372647,5474,344685
1,울산광역시,중구,162972,81207,45410,0,34682,80092,1115,81765
2,울산광역시,남구,234702,108730,63805,0,43804,107609,1121,125972


In [82]:
g3_ulsan_21.to_csv("temp2_1_governor_ulsan_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, X)

In [83]:
g3_ulsan_22 = rename_columns_by_index(g3_ulsan, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_박맹우' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [84]:
g3_ulsan_22 = add_sum_column(g3_ulsan_22, 5, 6, '그외정당')
g3_ulsan_22 = add_zero_column(g3_ulsan_22, '진보정당')
g3_ulsan_22 = add_zero_column(g3_ulsan_22, '무소속')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_3_민주노동당_송철호
6: 득표수_4_사회당_안승천

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [85]:
g3_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_민주노동당_송철호,득표수_4_사회당_안승천,득표수_계,무효투표수,기권수,그외정당,진보정당,무소속
0,울산광역시,합계,722806,378121,197772,162546,12329,372647,5474,344685,174875,0,0
1,울산광역시,중구,162972,81207,45410,32569,2113,80092,1115,81765,34682,0,0
2,울산광역시,남구,234702,108730,63805,40808,2996,107609,1121,125972,43804,0,0


In [86]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_ulsan_22 = g3_ulsan_22[selected_columns].copy()

g3_ulsan_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,울산광역시,합계,722806,378121,197772,0,174875,0,372647,5474,344685
1,울산광역시,중구,162972,81207,45410,0,34682,0,80092,1115,81765
2,울산광역시,남구,234702,108730,63805,0,43804,0,107609,1121,125972


In [87]:
g3_ulsan_22.to_csv("temp2_2_governor_ulsan_3.csv", index=False, encoding="utf-8-sig")

## Gyeonggi

In [88]:
g3_gyeonggi.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_손학규,득표수_2_새천년민주당_진념,득표수_3_민주노동당_김준기,득표수_계,무효투표수,기권수
0,경기도,합계,6777575,3024844,1744291,1075243,168357,2987891,36953,3752731
1,경기도,수원시장안구,233829,99253,59603,33422,5251,98276,977,134576
2,경기도,수원시권선구,225950,92598,55604,31178,4856,91638,960,133352
3,경기도,수원시팔달구,224122,88058,53550,29845,3859,87254,804,136064
4,경기도,성남시수정구,192566,78431,37715,34523,5176,77414,1017,114135


In [89]:
show_column_info(g3_gyeonggi)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_손학규
5: 득표수_2_새천년민주당_진념
6: 득표수_3_민주노동당_김준기
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 40


### v2.1

In [90]:
g3_gyeonggi_21 = rename_columns_by_index(g3_gyeonggi, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_손학규' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_진념' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [91]:
g3_gyeonggi_21 = add_sum_column(g3_gyeonggi_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주노동당_김준기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [92]:
g3_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_김준기,득표수_계,무효투표수,기권수,그외정당
0,경기도,합계,6777575,3024844,1744291,1075243,168357,2987891,36953,3752731,168357
1,경기도,수원시장안구,233829,99253,59603,33422,5251,98276,977,134576,5251
2,경기도,수원시권선구,225950,92598,55604,31178,4856,91638,960,133352,4856


In [93]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_gyeonggi_21 = g3_gyeonggi_21[selected_columns].copy()

g3_gyeonggi_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경기도,합계,6777575,3024844,1744291,1075243,168357,2987891,36953,3752731
1,경기도,수원시장안구,233829,99253,59603,33422,5251,98276,977,134576
2,경기도,수원시권선구,225950,92598,55604,31178,4856,91638,960,133352


In [94]:
g3_gyeonggi_21.to_csv("temp2_1_governor_gyeonggi_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [95]:
g3_gyeonggi_22 = rename_columns_by_index(g3_gyeonggi, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_손학규' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_진념' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [96]:
g3_gyeonggi_22 = add_sum_column(g3_gyeonggi_22, 6, 6, '그외정당')
g3_gyeonggi_22 = add_zero_column(g3_gyeonggi_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주노동당_김준기

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [97]:
g3_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_김준기,득표수_계,무효투표수,기권수,그외정당,무소속
0,경기도,합계,6777575,3024844,1744291,1075243,168357,2987891,36953,3752731,168357,0
1,경기도,수원시장안구,233829,99253,59603,33422,5251,98276,977,134576,5251,0
2,경기도,수원시권선구,225950,92598,55604,31178,4856,91638,960,133352,4856,0


In [98]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_gyeonggi_22 = g3_gyeonggi_22[selected_columns].copy()

g3_gyeonggi_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경기도,합계,6777575,3024844,1744291,1075243,168357,0,2987891,36953,3752731
1,경기도,수원시장안구,233829,99253,59603,33422,5251,0,98276,977,134576
2,경기도,수원시권선구,225950,92598,55604,31178,4856,0,91638,960,133352


In [99]:
g3_gyeonggi_22.to_csv("temp2_2_governor_gyeonggi_3.csv", index=False, encoding="utf-8-sig")

## Gangwon

In [100]:
g3_gangwon.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김진선,득표수_2_새천년민주당_남동우,득표수_계,무효투표수,기권수
0,강원도,합계,1129859,671242,468987,190451,659438,11804,458617
1,강원도,춘천시,181938,95277,52497,41426,93923,1354,86661
2,강원도,원주시,194677,97205,62405,32931,95336,1869,97472
3,강원도,강릉시,169090,91290,72430,17571,90001,1289,77800
4,강원도,동해시,74930,42530,35744,6107,41851,679,32400


In [101]:
show_column_info(g3_gangwon)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_김진선
5: 득표수_2_새천년민주당_남동우
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 19


### v2.1 (XX)

In [102]:
g3_gangwon_21 = rename_columns_by_index(g3_gangwon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김진선' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_남동우' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [103]:
g3_gangwon_21 = add_zero_column(g3_gangwon_21, '그외정당')

In [104]:
g3_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당
0,강원도,합계,1129859,671242,468987,190451,659438,11804,458617,0
1,강원도,춘천시,181938,95277,52497,41426,93923,1354,86661,0
2,강원도,원주시,194677,97205,62405,32931,95336,1869,97472,0


In [105]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_gangwon_21 = g3_gangwon_21[selected_columns].copy()

g3_gangwon_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,강원도,합계,1129859,671242,468987,190451,0,659438,11804,458617
1,강원도,춘천시,181938,95277,52497,41426,0,93923,1354,86661
2,강원도,원주시,194677,97205,62405,32931,0,95336,1869,97472


In [106]:
g3_gangwon_21.to_csv("temp2_1_governor_gangwon_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX, X)

In [107]:
g3_gangwon_22 = rename_columns_by_index(g3_gangwon, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김진선' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_남동우' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [108]:
g3_gangwon_22 = add_zero_column(g3_gangwon_22, '그외정당')
g3_gangwon_22 = add_zero_column(g3_gangwon_22, '무소속')

In [109]:
g3_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_계,무효투표수,기권수,그외정당,무소속
0,강원도,합계,1129859,671242,468987,190451,659438,11804,458617,0,0
1,강원도,춘천시,181938,95277,52497,41426,93923,1354,86661,0,0
2,강원도,원주시,194677,97205,62405,32931,95336,1869,97472,0,0


In [110]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_gangwon_22 = g3_gangwon_22[selected_columns].copy()

g3_gangwon_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,강원도,합계,1129859,671242,468987,190451,0,0,659438,11804,458617
1,강원도,춘천시,181938,95277,52497,41426,0,0,93923,1354,86661
2,강원도,원주시,194677,97205,62405,32931,0,0,95336,1869,97472


In [111]:
g3_gangwon_22.to_csv("temp2_2_governor_gangwon_3.csv", index=False, encoding="utf-8-sig")

## Chungbuk

In [112]:
g3_chungbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이원종,득표수_3_자유민주연합_구천서,득표수_4_무소속_장한량,득표수_계,무효투표수,기권수
0,충청북도,합계,1076451,600550,343546,196253,46459,586258,14292,475901
1,충청북도,청주시상당구,163686,76240,46565,21875,6693,75133,1107,87446
2,충청북도,청주시흥덕구,246912,109497,66724,32496,8636,107856,1641,137415
3,충청북도,충주시,154380,83817,49029,26735,5968,81732,2085,70563
4,충청북도,제천시,105294,62841,46428,11768,3476,61672,1169,42453


In [113]:
show_column_info(g3_chungbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_이원종
5: 득표수_3_자유민주연합_구천서
6: 득표수_4_무소속_장한량
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 13


### v2.1 (XXX)

In [114]:
g3_chungbuk_21 = rename_columns_by_index(g3_chungbuk, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_이원종' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [115]:
g3_chungbuk_21 = add_sum_column(g3_chungbuk_21, 5, 6, '그외정당')
g3_chungbuk_21 = add_zero_column(g3_chungbuk_21, '진보정당')


=== 합산할 열들 (인덱스 5~6) ===
5: 득표수_3_자유민주연합_구천서
6: 득표수_4_무소속_장한량

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [116]:
g3_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_자유민주연합_구천서,득표수_4_무소속_장한량,득표수_계,무효투표수,기권수,그외정당,진보정당
0,충청북도,합계,1076451,600550,343546,196253,46459,586258,14292,475901,242712,0
1,충청북도,청주시상당구,163686,76240,46565,21875,6693,75133,1107,87446,28568,0
2,충청북도,청주시흥덕구,246912,109497,66724,32496,8636,107856,1641,137415,41132,0


In [117]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_chungbuk_21 = g3_chungbuk_21[selected_columns].copy()

g3_chungbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청북도,합계,1076451,600550,343546,0,242712,586258,14292,475901
1,충청북도,청주시상당구,163686,76240,46565,0,28568,75133,1107,87446
2,충청북도,청주시흥덕구,246912,109497,66724,0,41132,107856,1641,137415


In [118]:
g3_chungbuk_21.to_csv("temp2_1_governor_chungbuk_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX)

In [119]:
g3_chungbuk_22 = rename_columns_by_index(g3_chungbuk, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_이원종' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [120]:
g3_chungbuk_22 = add_sum_column(g3_chungbuk_22, 5, 5, '그외정당')
g3_chungbuk_22 = add_sum_column(g3_chungbuk_22, 6, 6, '무소속')
g3_chungbuk_22 = add_zero_column(g3_chungbuk_22, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_자유민주연합_구천서

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11

=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_4_무소속_장한량

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 12


In [121]:
g3_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_자유민주연합_구천서,득표수_4_무소속_장한량,득표수_계,무효투표수,기권수,그외정당,무소속,진보정당
0,충청북도,합계,1076451,600550,343546,196253,46459,586258,14292,475901,196253,46459,0
1,충청북도,청주시상당구,163686,76240,46565,21875,6693,75133,1107,87446,21875,6693,0
2,충청북도,청주시흥덕구,246912,109497,66724,32496,8636,107856,1641,137415,32496,8636,0


In [122]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_chungbuk_22 = g3_chungbuk_22[selected_columns].copy()

g3_chungbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청북도,합계,1076451,600550,343546,0,196253,46459,586258,14292,475901
1,충청북도,청주시상당구,163686,76240,46565,0,21875,6693,75133,1107,87446
2,충청북도,청주시흥덕구,246912,109497,66724,0,32496,8636,107856,1641,137415


In [123]:
g3_chungbuk_22.to_csv("temp2_2_governor_chungbuk_3.csv", index=False, encoding="utf-8-sig")

## Chungnam

In [124]:
g3_chungnam.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_박태권,득표수_3_자유민주연합_심대평,득표수_계,무효투표수,기권수
0,충청남도,합계,1397105,785742,251005,508796,759801,25941,611363
1,충청남도,천안시,299432,126601,48169,74741,122910,3691,172831
2,충청남도,공주시,98876,63126,13237,48033,61270,1856,35750
3,충청남도,보령시,85349,54816,14094,39142,53236,1580,30533
4,충청남도,아산시,135721,67878,22147,43398,65545,2333,67843


In [125]:
show_column_info(g3_chungnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_박태권
5: 득표수_3_자유민주연합_심대평
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 16


### v2.1 (XXX)

In [126]:
g3_chungnam_21 = rename_columns_by_index(g3_chungnam, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_박태권' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [127]:
g3_chungnam_21 = add_sum_column(g3_chungnam_21, 5, 5, '그외정당')
g3_chungnam_21 = add_zero_column(g3_chungnam_21, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_자유민주연합_심대평

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [128]:
g3_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_자유민주연합_심대평,득표수_계,무효투표수,기권수,그외정당,진보정당
0,충청남도,합계,1397105,785742,251005,508796,759801,25941,611363,508796,0
1,충청남도,천안시,299432,126601,48169,74741,122910,3691,172831,74741,0
2,충청남도,공주시,98876,63126,13237,48033,61270,1856,35750,48033,0


In [129]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_chungnam_21 = g3_chungnam_21[selected_columns].copy()

g3_chungnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,충청남도,합계,1397105,785742,251005,0,508796,759801,25941,611363
1,충청남도,천안시,299432,126601,48169,0,74741,122910,3691,172831
2,충청남도,공주시,98876,63126,13237,0,48033,61270,1856,35750


In [130]:
g3_chungnam_21.to_csv("temp2_1_governor_chungnam_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, X)

In [131]:
g3_chungnam_22 = rename_columns_by_index(g3_chungnam, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_박태권' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [132]:
g3_chungnam_22 = add_sum_column(g3_chungnam_22, 5, 5, '그외정당')
g3_chungnam_22 = add_zero_column(g3_chungnam_22, '진보정당')
g3_chungnam_22 = add_zero_column(g3_chungnam_22, '무소속')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_자유민주연합_심대평

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [133]:
g3_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_자유민주연합_심대평,득표수_계,무효투표수,기권수,그외정당,진보정당,무소속
0,충청남도,합계,1397105,785742,251005,508796,759801,25941,611363,508796,0,0
1,충청남도,천안시,299432,126601,48169,74741,122910,3691,172831,74741,0,0
2,충청남도,공주시,98876,63126,13237,48033,61270,1856,35750,48033,0,0


In [134]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_chungnam_22 = g3_chungnam_22[selected_columns].copy()

g3_chungnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,충청남도,합계,1397105,785742,251005,0,508796,0,759801,25941,611363
1,충청남도,천안시,299432,126601,48169,0,74741,0,122910,3691,172831
2,충청남도,공주시,98876,63126,13237,0,48033,0,61270,1856,35750


In [135]:
g3_chungnam_22.to_csv("temp2_2_governor_chungnam_3.csv", index=False, encoding="utf-8-sig")

## Jeonbuk

In [136]:
g3_jeonbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_라경균,득표수_2_새천년민주당_강현욱,득표수_3_무소속_손주항,득표수_계,무효투표수,기권수
0,전라북도,합계,1431722,787135,63661,571650,131320,766631,20504,644587
1,전라북도,전주시완산구,231776,100204,8191,74023,15785,97999,2205,131572
2,전라북도,전주시덕진구,197458,79884,6453,59409,12470,78332,1552,117574
3,전라북도,군산시,197267,97041,5055,79646,10182,94883,2158,100226
4,전라북도,익산시,234964,112465,9121,84169,16164,109454,3011,122499


In [137]:
show_column_info(g3_jeonbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_라경균
5: 득표수_2_새천년민주당_강현욱
6: 득표수_3_무소속_손주항
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 16


### v2.1

In [138]:
g3_jeonbuk_21 = rename_columns_by_index(g3_jeonbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_라경균' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_강현욱' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [139]:
g3_jeonbuk_21 = add_sum_column(g3_jeonbuk_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_무소속_손주항

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [140]:
g3_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_무소속_손주항,득표수_계,무효투표수,기권수,그외정당
0,전라북도,합계,1431722,787135,63661,571650,131320,766631,20504,644587,131320
1,전라북도,전주시완산구,231776,100204,8191,74023,15785,97999,2205,131572,15785
2,전라북도,전주시덕진구,197458,79884,6453,59409,12470,78332,1552,117574,12470


In [141]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_jeonbuk_21 = g3_jeonbuk_21[selected_columns].copy()

g3_jeonbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라북도,합계,1431722,787135,63661,571650,131320,766631,20504,644587
1,전라북도,전주시완산구,231776,100204,8191,74023,15785,97999,2205,131572
2,전라북도,전주시덕진구,197458,79884,6453,59409,12470,78332,1552,117574


In [142]:
g3_jeonbuk_21.to_csv("temp2_1_governor_jeonbuk_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX)

In [143]:
g3_jeonbuk_22 = rename_columns_by_index(g3_jeonbuk, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_라경균' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_강현욱' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [144]:
g3_jeonbuk_22 = add_sum_column(g3_jeonbuk_22, 6, 6, '무소속')
g3_jeonbuk_22 = add_zero_column(g3_jeonbuk_22, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_무소속_손주항

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 11


In [145]:
g3_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_무소속_손주항,득표수_계,무효투표수,기권수,무소속,그외정당
0,전라북도,합계,1431722,787135,63661,571650,131320,766631,20504,644587,131320,0
1,전라북도,전주시완산구,231776,100204,8191,74023,15785,97999,2205,131572,15785,0
2,전라북도,전주시덕진구,197458,79884,6453,59409,12470,78332,1552,117574,12470,0


In [146]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_jeonbuk_22 = g3_jeonbuk_22[selected_columns].copy()

g3_jeonbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라북도,합계,1431722,787135,63661,571650,0,131320,766631,20504,644587
1,전라북도,전주시완산구,231776,100204,8191,74023,0,15785,97999,2205,131572
2,전라북도,전주시덕진구,197458,79884,6453,59409,0,12470,78332,1552,117574


In [147]:
g3_jeonbuk_22.to_csv("temp2_2_governor_jeonbuk_3.csv", index=False, encoding="utf-8-sig")

## Jeonnam

In [148]:
g3_jeonnam.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_황수연,득표수_2_새천년민주당_박태영,득표수_3_무소속_송재구,득표수_4_무소속_송하성,득표수_5_무소속_안수원,득표수_계,무효투표수,기권수
0,전라남도,합계,1533059,1006148,51504,563545,236558,89487,34439,975533,30615,526911
1,전라남도,목포시,169428,81949,2234,48314,24124,4459,1012,80143,1806,87479
2,전라남도,여수시,224680,128464,5123,73429,32229,11837,2700,125318,3146,96216
3,전라남도,순천시,185581,102005,4062,55386,22295,13613,3792,99148,2857,83576
4,전라남도,나주시,80868,56762,2914,33315,9992,6442,2194,54857,1905,24106


In [149]:
show_column_info(g3_jeonnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_황수연
5: 득표수_2_새천년민주당_박태영
6: 득표수_3_무소속_송재구
7: 득표수_4_무소속_송하성
8: 득표수_5_무소속_안수원
9: 득표수_계
10: 무효투표수
11: 기권수

총 열 개수: 12
총 행 개수: 23


### v2.1

In [150]:
g3_jeonnam_21 = rename_columns_by_index(g3_jeonnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_황수연' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_박태영' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [151]:
g3_jeonnam_21 = add_sum_column(g3_jeonnam_21, 6, 8, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_무소속_송재구
7: 득표수_4_무소속_송하성
8: 득표수_5_무소속_안수원

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 13


In [152]:
g3_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_무소속_송재구,득표수_4_무소속_송하성,득표수_5_무소속_안수원,득표수_계,무효투표수,기권수,그외정당
0,전라남도,합계,1533059,1006148,51504,563545,236558,89487,34439,975533,30615,526911,360484
1,전라남도,목포시,169428,81949,2234,48314,24124,4459,1012,80143,1806,87479,29595
2,전라남도,여수시,224680,128464,5123,73429,32229,11837,2700,125318,3146,96216,46766


In [153]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_jeonnam_21 = g3_jeonnam_21[selected_columns].copy()

g3_jeonnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,전라남도,합계,1533059,1006148,51504,563545,360484,975533,30615,526911
1,전라남도,목포시,169428,81949,2234,48314,29595,80143,1806,87479
2,전라남도,여수시,224680,128464,5123,73429,46766,125318,3146,96216


In [154]:
g3_jeonnam_21.to_csv("temp2_1_governor_jeonnam_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XX)

In [155]:
g3_jeonnam_22 = rename_columns_by_index(g3_jeonnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_황수연' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_박태영' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [156]:
g3_jeonnam_22 = add_sum_column(g3_jeonnam_22, 6, 8, '무소속')
g3_jeonnam_22 = add_zero_column(g3_jeonnam_22, '그외정당')


=== 합산할 열들 (인덱스 6~8) ===
6: 득표수_3_무소속_송재구
7: 득표수_4_무소속_송하성
8: 득표수_5_무소속_안수원

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 13


In [157]:
g3_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_무소속_송재구,득표수_4_무소속_송하성,득표수_5_무소속_안수원,득표수_계,무효투표수,기권수,무소속,그외정당
0,전라남도,합계,1533059,1006148,51504,563545,236558,89487,34439,975533,30615,526911,360484,0
1,전라남도,목포시,169428,81949,2234,48314,24124,4459,1012,80143,1806,87479,29595,0
2,전라남도,여수시,224680,128464,5123,73429,32229,11837,2700,125318,3146,96216,46766,0


In [158]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_jeonnam_22 = g3_jeonnam_22[selected_columns].copy()

g3_jeonnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,전라남도,합계,1533059,1006148,51504,563545,0,360484,975533,30615,526911
1,전라남도,목포시,169428,81949,2234,48314,0,29595,80143,1806,87479
2,전라남도,여수시,224680,128464,5123,73429,0,46766,125318,3146,96216


In [159]:
g3_jeonnam_22.to_csv("temp2_2_governor_jeonnam_3.csv", index=False, encoding="utf-8-sig")

## Gyeongbuk

In [160]:
g3_gyeongbuk.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_이의근,득표수_3_무소속_조영건,득표수_계,무효투표수,기권수
0,경상북도,합계,2044215,1234095,1028080,174472,1202552,31543,810120
1,경상북도,포항시북구,178954,95784,82300,11615,93915,1869,83170
2,경상북도,포항시남구,186347,97807,82806,13093,95899,1908,88540
3,경상북도,울릉군,7630,6638,5598,917,6515,123,992
4,경상북도,경주시,210873,125634,107442,15084,122526,3108,85239


In [161]:
show_column_info(g3_gyeongbuk)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_이의근
5: 득표수_3_무소속_조영건
6: 득표수_계
7: 무효투표수
8: 기권수

총 열 개수: 9
총 행 개수: 25


### v2.1 (XXX)

In [162]:
g3_gyeongbuk_21 = rename_columns_by_index(g3_gyeongbuk, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_이의근' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [163]:
g3_gyeongbuk_21 = add_sum_column(g3_gyeongbuk_21, 5, 5, '그외정당')
g3_gyeongbuk_21 = add_zero_column(g3_gyeongbuk_21, '진보정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_무소속_조영건

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 10


In [164]:
g3_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_무소속_조영건,득표수_계,무효투표수,기권수,그외정당,진보정당
0,경상북도,합계,2044215,1234095,1028080,174472,1202552,31543,810120,174472,0
1,경상북도,포항시북구,178954,95784,82300,11615,93915,1869,83170,11615,0
2,경상북도,포항시남구,186347,97807,82806,13093,95899,1908,88540,13093,0


In [165]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_gyeongbuk_21 = g3_gyeongbuk_21[selected_columns].copy()

g3_gyeongbuk_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상북도,합계,2044215,1234095,1028080,0,174472,1202552,31543,810120
1,경상북도,포항시북구,178954,95784,82300,0,11615,93915,1869,83170
2,경상북도,포항시남구,186347,97807,82806,0,13093,95899,1908,88540


In [166]:
g3_gyeongbuk_21.to_csv("temp2_1_governor_gyeongbuk_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (XXX, XX)

In [167]:
g3_gyeongbuk_22 = rename_columns_by_index(g3_gyeongbuk, 4, '보수정당')

인덱스 4: '득표수_1_한나라당_이의근' -> '보수정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 1


In [168]:
g3_gyeongbuk_22 = add_sum_column(g3_gyeongbuk_22, 5, 5, '무소속')
g3_gyeongbuk_22 = add_zero_column(g3_gyeongbuk_22, '진보정당')
g3_gyeongbuk_22 = add_zero_column(g3_gyeongbuk_22, '그외정당')


=== 합산할 열들 (인덱스 5~5) ===
5: 득표수_3_무소속_조영건

=== 새로운 열 추가 완료 ===
새로운 열 이름: 무소속
총 열 개수: 10


In [169]:
g3_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,득표수_3_무소속_조영건,득표수_계,무효투표수,기권수,무소속,진보정당,그외정당
0,경상북도,합계,2044215,1234095,1028080,174472,1202552,31543,810120,174472,0,0
1,경상북도,포항시북구,178954,95784,82300,11615,93915,1869,83170,11615,0,0
2,경상북도,포항시남구,186347,97807,82806,13093,95899,1908,88540,13093,0,0


In [170]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_gyeongbuk_22 = g3_gyeongbuk_22[selected_columns].copy()

g3_gyeongbuk_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상북도,합계,2044215,1234095,1028080,0,0,174472,1202552,31543,810120
1,경상북도,포항시북구,178954,95784,82300,0,0,11615,93915,1869,83170
2,경상북도,포항시남구,186347,97807,82806,0,0,13093,95899,1908,88540


In [171]:
g3_gyeongbuk_22.to_csv("temp2_2_governor_gyeongbuk_3.csv", index=False, encoding="utf-8-sig")

## Gyeongnam

In [172]:
g3_gyeongnam.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_김혁규,득표수_2_새천년민주당_김두관,득표수_3_민주노동당_임수태,득표수_계,무효투표수,기권수
0,경상남도,합계,2227548,1259142,920706,208641,106367,1235714,23428,968406
1,경상남도,창원시,343825,166981,111414,29630,24139,165183,1798,176844
2,경상남도,마산시,316267,153345,118410,20247,12621,151278,2067,162922
3,경상남도,진주시,242187,143459,108529,23928,8679,141136,2323,98728
4,경상남도,진해시,99953,60998,47057,8519,4555,60131,867,38955


In [173]:
show_column_info(g3_gyeongnam)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_김혁규
5: 득표수_2_새천년민주당_김두관
6: 득표수_3_민주노동당_임수태
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 21


### v2.1

In [174]:
g3_gyeongnam_21 = rename_columns_by_index(g3_gyeongnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김혁규' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_김두관' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [175]:
g3_gyeongnam_21 = add_sum_column(g3_gyeongnam_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주노동당_임수태

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [176]:
g3_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_임수태,득표수_계,무효투표수,기권수,그외정당
0,경상남도,합계,2227548,1259142,920706,208641,106367,1235714,23428,968406,106367
1,경상남도,창원시,343825,166981,111414,29630,24139,165183,1798,176844,24139
2,경상남도,마산시,316267,153345,118410,20247,12621,151278,2067,162922,12621


In [177]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_gyeongnam_21 = g3_gyeongnam_21[selected_columns].copy()

g3_gyeongnam_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,경상남도,합계,2227548,1259142,920706,208641,106367,1235714,23428,968406
1,경상남도,창원시,343825,166981,111414,29630,24139,165183,1798,176844
2,경상남도,마산시,316267,153345,118410,20247,12621,151278,2067,162922


In [178]:
g3_gyeongnam_21.to_csv("temp2_1_governor_gyeongnam_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [179]:
g3_gyeongnam_22 = rename_columns_by_index(g3_gyeongnam, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_김혁규' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_김두관' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [180]:
g3_gyeongnam_22 = add_sum_column(g3_gyeongnam_22, 6, 6, '그외정당')
g3_gyeongnam_22 = add_zero_column(g3_gyeongnam_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주노동당_임수태

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [181]:
g3_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주노동당_임수태,득표수_계,무효투표수,기권수,그외정당,무소속
0,경상남도,합계,2227548,1259142,920706,208641,106367,1235714,23428,968406,106367,0
1,경상남도,창원시,343825,166981,111414,29630,24139,165183,1798,176844,24139,0
2,경상남도,마산시,316267,153345,118410,20247,12621,151278,2067,162922,12621,0


In [182]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_gyeongnam_22 = g3_gyeongnam_22[selected_columns].copy()

g3_gyeongnam_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,경상남도,합계,2227548,1259142,920706,208641,106367,0,1235714,23428,968406
1,경상남도,창원시,343825,166981,111414,29630,24139,0,165183,1798,176844
2,경상남도,마산시,316267,153345,118410,20247,12621,0,151278,2067,162922


In [183]:
g3_gyeongnam_22.to_csv("temp2_2_governor_gyeongnam_3.csv", index=False, encoding="utf-8-sig")

## Jeju

In [184]:
g3_jeju.head()

,시도,구시군,선거인수,투표수,득표수_1_한나라당_신구범,득표수_2_새천년민주당_우근민,득표수_3_민주국민당_신두완,득표수_계,무효투표수,기권수
0,제주도,합계,387982,267479,119502,135283,8373,263158,4321,120503
1,제주도,제주시,196356,124337,57387,62036,3402,122825,1512,72019
2,제주도,북제주군,74211,55474,24939,27484,1734,54157,1317,18737
3,제주도,서귀포시,60910,44602,19471,22759,1690,43920,682,16308
4,제주도,남제주군,56505,43066,17705,23004,1547,42256,810,13439


In [185]:
show_column_info(g3_jeju)

=== 열 이름과 인덱스 ===
0: 시도
1: 구시군
2: 선거인수
3: 투표수
4: 득표수_1_한나라당_신구범
5: 득표수_2_새천년민주당_우근민
6: 득표수_3_민주국민당_신두완
7: 득표수_계
8: 무효투표수
9: 기권수

총 열 개수: 10
총 행 개수: 5


### v2.1

In [186]:
g3_jeju_21 = rename_columns_by_index(g3_jeju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_신구범' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_우근민' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [187]:
g3_jeju_21 = add_sum_column(g3_jeju_21, 6, 6, '그외정당')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주국민당_신두완

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [188]:
g3_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주국민당_신두완,득표수_계,무효투표수,기권수,그외정당
0,제주도,합계,387982,267479,119502,135283,8373,263158,4321,120503,8373
1,제주도,제주시,196356,124337,57387,62036,3402,122825,1512,72019,3402
2,제주도,북제주군,74211,55474,24939,27484,1734,54157,1317,18737,1734


In [189]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '득표수_계', '무효투표수', '기권수']

g3_jeju_21 = g3_jeju_21[selected_columns].copy()

g3_jeju_21.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,득표수_계,무효투표수,기권수
0,제주도,합계,387982,267479,119502,135283,8373,263158,4321,120503
1,제주도,제주시,196356,124337,57387,62036,3402,122825,1512,72019
2,제주도,북제주군,74211,55474,24939,27484,1734,54157,1317,18737


In [190]:
g3_jeju_21.to_csv("temp2_1_governor_jeju_3.csv", index=False, encoding="utf-8-sig")

### v2.2 (X)

In [191]:
g3_jeju_22 = rename_columns_by_index(g3_jeju, [4, 5], ['보수정당', '진보정당'])

인덱스 4: '득표수_1_한나라당_신구범' -> '보수정당'
인덱스 5: '득표수_2_새천년민주당_우근민' -> '진보정당'

=== 열 이름 변경 완료 ===
변경된 열 개수: 2


In [192]:
g3_jeju_22 = add_sum_column(g3_jeju_22, 6, 6, '그외정당')
g3_jeju_22 = add_zero_column(g3_jeju_22, '무소속')


=== 합산할 열들 (인덱스 6~6) ===
6: 득표수_3_민주국민당_신두완

=== 새로운 열 추가 완료 ===
새로운 열 이름: 그외정당
총 열 개수: 11


In [193]:
g3_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,득표수_3_민주국민당_신두완,득표수_계,무효투표수,기권수,그외정당,무소속
0,제주도,합계,387982,267479,119502,135283,8373,263158,4321,120503,8373,0
1,제주도,제주시,196356,124337,57387,62036,3402,122825,1512,72019,3402,0
2,제주도,북제주군,74211,55474,24939,27484,1734,54157,1317,18737,1734,0


In [194]:
# 원하는 열들을 리스트로 지정 후 선택
selected_columns = ['시도', '구시군', '선거인수', '투표수', '보수정당', '진보정당', '그외정당', '무소속', '득표수_계', '무효투표수', '기권수']

g3_jeju_22 = g3_jeju_22[selected_columns].copy()

g3_jeju_22.head(3)

,시도,구시군,선거인수,투표수,보수정당,진보정당,그외정당,무소속,득표수_계,무효투표수,기권수
0,제주도,합계,387982,267479,119502,135283,8373,0,263158,4321,120503
1,제주도,제주시,196356,124337,57387,62036,3402,0,122825,1512,72019
2,제주도,북제주군,74211,55474,24939,27484,1734,0,54157,1317,18737


In [195]:
g3_jeju_22.to_csv("temp2_2_governor_jeju_3.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [196]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp2_2_governor_gyeongnam_3.csv
Added: temp2_1_governor_jeju_3.csv
Added: temp2_2_governor_gyeonggi_3.csv
Added: temp2_1_governor_incheon_3.csv
Added: temp2_1_governor_jeonbuk_3.csv
Added: temp2_1_governor_daegu_3.csv
Added: temp2_1_governor_ulsan_3.csv
Added: temp2_2_governor_chungbuk_3.csv
Added: temp2_2_governor_jeonbuk_3.csv
Added: temp2_2_governor_daegu_3.csv
Added: temp2_2_governor_busan_3.csv
Added: temp2_1_governor_gwangju_3.csv
Added: temp2_2_governor_ulsan_3.csv
Added: temp2_2_governor_seoul_3.csv
Added: temp2_1_governor_gyeonggi_3.csv
Added: temp2_2_governor_jeonnam_3.csv
Added: temp2_2_governor_gyeongbuk_3.csv
Added: temp2_1_governor_gangwon_3.csv
Added: temp2_1_governor_gyeongbuk_3.csv
Added: temp2_1_governor_chungbuk_3.csv
Added: temp2_2_governor_chungnam_3.csv
Added: temp2_1_governor_seoul_3.csv
Added: temp2_1_governor_chungnam_3.csv
Added: temp2_1_governor_daejeon_3.csv
Added: temp2_2_governor_daejeon_3.csv
Added: temp2_1_governor_gyeongnam_3.csv
Added: temp2_2_